In [4]:
import json
import folium
from shapely.geometry import Point, Polygon, LineString, shape

# Load the Pontaise limits GeoJSON
with open('Pontaise_limits.geojson', 'r') as f:
    pontaise_limits = json.load(f)

# Load the numeros merged GeoJSON
with open('numeros_merged-v3.geojson', 'r') as f:
    numeros_data = json.load(f)

# Extract the LineString from Pontaise_limits and convert to Polygon
linestring_geom = shape(pontaise_limits['features'][0]['geometry'])
# Convert LineString to Polygon by closing it
pontaise_polygon = Polygon(linestring_geom.coords)

print(f"Pontaise polygon created with {len(pontaise_polygon.exterior.coords)} vertices")

# Filter points that are inside the Pontaise boundary
filtered_features = []
for feature in numeros_data['features']:
    point = Point(feature['geometry']['coordinates'])
    if pontaise_polygon.contains(point):
        filtered_features.append(feature)

print(f"Filtered {len(filtered_features)} points inside Pontaise boundary")

# Create new GeoJSON with filtered points
pontaise_points = {
    "type": "FeatureCollection",
    "name": "pontaise_points",
    "crs": numeros_data.get('crs'),
    "features": filtered_features
}

# Save to new GeoJSON file
with open('pontaise_points.geojson', 'w') as f:
    json.dump(pontaise_points, f, indent=2)

print(f"Saved to pontaise_points.geojson")

# Create Folium map centered on the data
if filtered_features:
    # Calculate center of the polygon
    center_lat = pontaise_polygon.centroid.y
    center_lon = pontaise_polygon.centroid.x
else:
    center_lat, center_lon = 46.528, 6.630

# Create map
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add the Pontaise boundary polygon
folium.GeoJson(
    pontaise_limits,
    name='Pontaise Boundary',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 3,
        'fillOpacity': 0.1
    }
).add_to(m)

# Add the filtered points
for feature in filtered_features:
    coords = feature['geometry']['coordinates']
    properties = feature['properties']
    
    # Create popup text with properties
    popup_text = "<br>".join([f"<b>{k}:</b> {v}" for k, v in properties.items()])
    
    folium.CircleMarker(
        location=[coords[1], coords[0]],  # lat, lon
        radius=3,
        popup=folium.Popup(popup_text, max_width=300),
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

Pontaise polygon created with 51 vertices
Filtered 274 points inside Pontaise boundary
Saved to pontaise_points.geojson
